In [1]:
import pandas as pd
import datetime
import sys, os
import numpy as np
from os import path
from pathlib import Path
from binance.spot import Spot
from twelvedata import TDClient
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import itertools
import xgboost as xgb
import seaborn as sns
from typing import Union, Dict, List
import talib
import ta
import shap
from coinmetrics.api_client import CoinMetricsClient
sys.path.insert(0, os.path.join(os.getcwd(), ".."))
import fin_utilities 
import sklearn
import scikitplot as skplt
import matplotlib.pyplot as plt
cfg = fin_utilities.__cfg_reading("pred")
import my_functions
import tensorflow as tf
import matplotlib as mpl
# Set the default color cycle
mpl.rcParams.update(mpl.rcParamsDefault)


#td = TDClient(apikey=cfg['TWELVEDATA']['API'])  

PROJECT_DIR = eval(cfg['PROJECT_PATH'])
DATA_PATH   = PROJECT_DIR / cfg['DATA_FOLDER']
SOURCE      = cfg['API_DATA_SOURCE']
URL         = cfg[SOURCE]['API_URL_HIST_DATA']
COLUMNS     = cfg[SOURCE]['COLUMN_NAMES']
PARAMS      = cfg[SOURCE]['REQ_PARAMS']
SYMBOL      = cfg['SYMBOL']
STABLECOIN  = cfg['STABLECOIN']
PAIR        = SYMBOL + STABLECOIN #binance
MAX_LENGTH  = cfg['MAX_TRADE_DURATION']
XGB_PARAM   = cfg['xgb']

c:\Users\ma.orlandi\Anaconda3\envs\obb\lib\site-packages\torch\onnx\_internal\_beartype.py:30: UserWarning: module 'beartype.roar' has no attribute 'BeartypeDecorHintPep585DeprecationWarning'
  warnings.warn(f"{e}")


In [2]:
import requests

import requests

def get_historical_active_addresses(api_key, start_date, end_date):
    base_url = "https://api.etherscan.io/api"
    endpoint = "/gastracker"
    module = "gastracker"
    action = "getHistoricalAvg"
    
    url = f"{base_url}?module={module}&action={action}&startblock=0&endblock=999999999&apikey={api_key}"

    try:
        response = requests.get(url)
        data = response.json()

        if data["status"] == "1":
            historical_active_addresses = data["result"]
            return historical_active_addresses
        else:
            print("Error:", data["message"])
            return None

    except requests.exceptions.RequestException as e:
        print("Error making API request:", e)
        return None

# Replace 'YOUR_API_KEY' with your actual Etherscan API key
api_key = "13ZJICS19X7YDPTA52GQXH1AU31MC187V1"

# Replace 'start_date' and 'end_date' with the desired date range (in UNIX timestamp format)
start_date = 1609459200  # Example: January 1, 2021
end_date = 1625097600    # Example: June 30, 2021

historical_active_addresses = get_historical_active_addresses(api_key, start_date, end_date)

if historical_active_addresses is not None:
    print("Historical Active Addresses:")
    print(historical_active_addresses)


Error: NOTOK


In [21]:
import yfinance as yf

ticker = "BTC-USD"
interval = "1h"

# Define the start and end dates
end_date = "2023-07-19"
start_date = (pd.to_datetime(end_date) - pd.Timedelta(hours=700)).strftime("%Y-%m-%d")

# Download the first chunk of historical data
data = yf.download(ticker, start=start_date, end=end_date, interval=interval)

# Download additional chunks of historical data if needed
while start_date > "2009-01-01":
    end_date = start_date
    start_date = (pd.to_datetime(start_date) - pd.Timedelta(hours=700)).strftime("%Y-%m-%d")
    chunk = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    data = pd.concat([chunk, data])

# Print the data
data = data.reset_index()
data.columns = ['Time','Open','High','Low','Close','Adj_Close','Volume']
data.to_csv(DATA_PATH / f'{ticker}_{interval}.csv',index=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [34]:
df = data.copy()
df.drop(columns='Adj_Close',inplace=True)

In [35]:
df

,Time,Open,High,Low,Close,Volume
0,2021-08-28 00:00:00,49044.132812,49264.675781,48902.218750,49238.804688,0.0
1,2021-08-28 01:00:00,49247.457031,49283.503906,49092.316406,49130.039062,9052160.0
2,2021-08-28 02:00:00,49132.082031,49132.082031,48878.121094,48972.496094,173776896.0
3,2021-08-28 03:00:00,48981.972656,49049.718750,48896.222656,48911.859375,0.0
4,2021-08-28 04:00:00,48969.011719,49099.828125,48969.011719,49030.125000,0.0
...,...,...,...,...,...,...
16394,2023-07-18 19:00:00,29909.402344,29936.634766,29710.734375,29720.015625,0.0
16395,2023-07-18 20:00:00,29719.406250,29794.064453,29704.521484,29777.044922,0.0
16396,2023-07-18 21:00:00,29776.722656,29833.054688,29764.291016,29800.257812,0.0
16397,2023-07-18 22:00:00,29797.849609,29823.927734,29770.392578,29821.093750,0.0


In [36]:
#TENTATIVO DI NORMALIZZAZIONE DEI DATI
df['Delta_Open_PrevClose'] = df['Open']-df['Close'].shift()
df['High'] = df['High'] - df['Open'] + df['Delta_Open_PrevClose']
df['Low'] = df['Low'] - df['Open'] + df['Delta_Open_PrevClose']
df['Close'] = df['Close'] - df['Open'] + df['Delta_Open_PrevClose']
df['Open'] = df['Delta_Open_PrevClose']
#df = df.drop(columns='Delta_Open_PrevClose')
df = df.dropna().reset_index(drop=True)
df

,Time,Open,High,Low,Close,Volume,Delta_Open_PrevClose
0,2021-08-28 01:00:00,8.652344,44.699219,-146.488281,-108.765625,9052160.0,8.652344
1,2021-08-28 02:00:00,2.042969,2.042969,-251.917969,-157.542969,173776896.0,2.042969
2,2021-08-28 03:00:00,9.476562,77.222656,-76.273438,-60.636719,0.0,9.476562
3,2021-08-28 04:00:00,57.152344,187.968750,57.152344,118.265625,0.0,57.152344
4,2021-08-28 05:00:00,-42.207031,24.148438,-92.578125,-71.562500,0.0,-42.207031
...,...,...,...,...,...,...,...
16393,2023-07-18 19:00:00,7.435547,34.667969,-191.232422,-181.951172,0.0,7.435547
16394,2023-07-18 20:00:00,-0.609375,74.048828,-15.494141,57.029297,0.0,-0.609375
16395,2023-07-18 21:00:00,-0.322266,56.009766,-12.753906,23.212891,0.0,-0.322266
16396,2023-07-18 22:00:00,-2.408203,23.669922,-29.865234,20.835938,0.0,-2.408203


In [75]:
map_ohlc = {'Open':'Open','High':'High','Low':'Low','Close':'Close'}
lags_relevant = [13,30,99,200]
#lags_relevant = [10]

In [76]:
def transform_percentages(df, columns, suffix='_pct'):

    df_perc = df.copy()

    for col in columns:
        df_perc[col+suffix] = np.round(100 * (df_perc[col]-df_perc[col].shift()) / df_perc[col].shift(),3)

    return df_perc

df = transform_percentages(df,map_ohlc)

In [77]:
df['green_candle']=np.where(df[map_ohlc['Open']]<df[map_ohlc['Close']],1,0)
#conto le candele consecutive verdi
df['cons_green_candles'] = df['green_candle'].groupby((df['green_candle'] == 0).cumsum()).cumsum() - 1 
#setto a 0 se ci sono -1
df['cons_green_candles'] = df['cons_green_candles'].clip(lower=0)
#conto le ultime n = 24 candele ad 1
df['rolling_count'] = df['green_candle'].rolling(window=24, min_periods=1).sum() -1 
df['rolling_count'] = df['rolling_count'].clip(lower=0)

df = df.drop(columns=['green_candle'])

In [78]:
print(df.columns)
df['AVG_Candle_Price'] = (df[map_ohlc['Open']]+df[map_ohlc['High']]+df[map_ohlc['Low']]+df[map_ohlc['Close']])/4
for lag in lags_relevant:
    df['Close_lag_'+str(lag)+'d'] = df['AVG_Candle_Price'].shift(lag)
    df['SMA_Close_'+str(lag)+'d'] = my_functions.rolling_kpi(df,'AVG_Candle_Price',lag,'mean', False)
    df['STD_Close_'+str(lag)+'d'] = my_functions.rolling_kpi(df,'AVG_Candle_Price',lag,'std', False)
    df['VAR_Close_lag_'+str(lag)+'d'] = df['STD_Close_'+str(lag)+'d']**2
    df['diff_Close_'+str(lag)+'d'] = df['AVG_Candle_Price'] - df['SMA_Close_'+str(lag)+'d']


Index(['Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open_pct',
       'High_pct', 'Low_pct', 'Close_pct', 'cons_green_candles',
       'rolling_count'],
      dtype='object')


In [79]:
df.tail(10)

,Time,Open,High,Low,Close,Volume,Open_pct,High_pct,Low_pct,Close_pct,...,Close_lag_99d,SMA_Close_99d,STD_Close_99d,VAR_Close_lag_99d,diff_Close_99d,Close_lag_200d,SMA_Close_200d,STD_Close_200d,VAR_Close_lag_200d,diff_Close_200d
17080,2023-07-18 14:00:00,29827.253906,30004.337891,29827.253906,29917.933594,415912960.0,0.157,0.501,0.297,0.448,...,31197.346680,30285.607052,269.086661,72407.631283,-391.412228,30088.257812,30467.517727,370.059377,136943.942704,-573.322903
17081,2023-07-18 15:00:00,29924.228516,29924.228516,29843.468750,29909.234375,0.0,0.325,-0.267,0.054,-0.029,...,31180.748535,30272.673128,256.080692,65577.320797,-372.383089,30108.770020,30466.475327,371.366312,137912.937744,-566.185288
17082,2023-07-18 16:00:00,29904.332031,29956.478516,29556.427734,29715.876953,379166720.0,-0.066,0.108,-0.962,-0.646,...,31198.817383,30258.374758,243.027276,59062.256767,-475.095950,30090.911133,30464.937166,373.560288,139547.288651,-681.658357
17083,2023-07-18 17:00:00,29711.855469,29799.978516,29683.798828,29799.978516,0.0,-0.644,-0.522,0.431,0.283,...,31286.614258,30242.842320,225.119060,50678.591124,-493.939488,30119.644043,30463.083459,376.192917,141521.110667,-714.180627
17084,2023-07-18 18:00:00,29820.083984,29919.382812,29794.798828,29901.966797,0.0,0.364,0.401,0.374,0.342,...,31289.622070,30228.392179,201.964145,40789.515875,-369.334073,30181.418457,30461.471658,378.091564,142953.230635,-602.413552
17085,2023-07-18 19:00:00,29909.402344,29936.634766,29710.734375,29720.015625,0.0,0.300,0.058,-0.282,-0.608,...,31216.342773,30214.279593,179.825588,32337.241956,-395.082815,30210.437500,30459.515454,380.401972,144705.660419,-640.318677
17086,2023-07-18 20:00:00,29719.406250,29794.064453,29704.521484,29777.044922,0.0,-0.635,-0.476,-0.021,0.192,...,30985.184570,30201.790448,168.305556,28326.760178,-453.031171,30204.938965,30457.234556,383.292737,146913.321999,-708.475278
17087,2023-07-18 21:00:00,29776.722656,29833.054688,29764.291016,29800.257812,0.0,0.193,0.131,0.201,0.078,...,30418.885742,30195.474244,171.772782,29505.888698,-401.892701,30217.180176,30455.116562,385.788126,148832.477964,-661.535020
17088,2023-07-18 22:00:00,29797.849609,29823.927734,29770.392578,29821.093750,0.0,0.071,-0.031,0.020,0.070,...,30114.861328,30192.327321,176.064877,30998.840751,-389.011403,30268.398926,30452.791147,388.312689,150786.744764,-649.475229
17089,2023-07-18 23:00:00,29822.896484,29857.214844,29795.808594,29857.214844,0.0,0.084,0.112,0.085,0.121,...,30146.930664,30189.159170,179.674995,32283.103920,-355.875478,30293.732910,30450.488901,390.618580,152582.874904,-617.205210


In [80]:
def return_index_if_exists(df_series, curr_idx, val, pos_crit, max_length):
    
    current_val = val

    if pos_crit:  

        try:
            
            thing_index = list(el >= current_val for el in df_series[curr_idx+1:]).index(True) +1
            #print(current_val)
            #print(list(df_series[curr_idx+1:]))
            #print(list(el >= current_val for el in df_series[curr_idx+1:]))
            #print(thing_index)
        except ValueError:
            thing_index = max_length
    else:

        try:
            
            thing_index = list(el <= current_val for el in df_series[curr_idx+1:]).index(True) +1
        except ValueError:
            thing_index = max_length

    return thing_index

def labelize_output_according_criterion2(df: pd.DataFrame, map_ohlc:dict, wrt:str='Close', target_var:str='signal', threshold:float=0.015, risk_reward_ratio:float=.5, max_trade_length:int=5) -> pd.DataFrame:
    
    df_ = df.copy()
    positive_criterion = 1 + threshold/risk_reward_ratio
    negative_criterion = 1 - threshold
    
    min_above = []
    min_below = []

    df_['TP'] = df_[map_ohlc[wrt]]*positive_criterion
    df_['SL'] = df_[map_ohlc[wrt]]*negative_criterion

    for idx,row in df_.iterrows():

        pos_val = positive_criterion*row[map_ohlc[wrt]]
        neg_val = negative_criterion*row[map_ohlc[wrt]]

        if idx != df_.index[-1]:
            candidates_above_minima = []
            candidates_below_minima = []
            for _,v in map_ohlc.items():
                candidates_above_minima.append(return_index_if_exists(df_[v], idx, pos_val, True, max_trade_length))
                candidates_below_minima.append(return_index_if_exists(df_[v], idx, neg_val, False, max_trade_length))

            min_above.append(min(candidates_above_minima))
            min_below.append(min(candidates_below_minima))

    min_above.append(None)
    min_below.append(None)

    df_['min_above'] = min_above
    df_['min_below'] = min_below

    df_[target_var] = (df_['min_above'].lt(df_['min_below'])) & (df_['min_above'] <= max_trade_length)
    df_[target_var]=df_[target_var].astype(int)

    df_ = df_.drop(columns=['min_above','min_below'])

    return df_


In [81]:
df.head()

,Time,Open,High,Low,Close,Volume,Open_pct,High_pct,Low_pct,Close_pct,...,Close_lag_99d,SMA_Close_99d,STD_Close_99d,VAR_Close_lag_99d,diff_Close_99d,Close_lag_200d,SMA_Close_200d,STD_Close_200d,VAR_Close_lag_200d,diff_Close_200d
0,2021-07-29 00:00:00,39995.453125,40200.531250,39620.410156,39647.917969,0.0,NaN,NaN,NaN,NaN,...,NaN,39866.078125,0.000000,0.000000,0.000000,NaN,39866.078125,0.000000,0.000000,0.000000
1,2021-07-29 01:00:00,39604.121094,39728.531250,39352.058594,39685.195312,0.0,-0.978,-1.174,-0.677,0.094,...,NaN,39729.277344,193.465520,37428.907501,-136.800781,NaN,39729.277344,193.465520,37428.907501,-136.800781
2,2021-07-29 02:00:00,39684.011719,39942.167969,39655.148438,39764.554688,0.0,0.202,0.538,0.770,0.200,...,NaN,39740.008464,138.057686,19059.924547,21.462240,NaN,39740.008464,138.057686,19059.924547,21.462240
3,2021-07-29 03:00:00,39727.890625,39855.910156,39574.535156,39700.894531,0.0,0.111,-0.216,-0.203,-0.160,...,NaN,39733.708252,113.425689,12865.387029,-18.900635,NaN,39733.708252,113.425689,12865.387029,-18.900635
4,2021-07-29 04:00:00,39640.609375,40110.203125,39579.765625,39983.476562,4128768.0,-0.220,0.638,0.013,0.712,...,NaN,39752.669336,106.989036,11446.653801,75.844336,NaN,39752.669336,106.989036,11446.653801,75.844336


In [83]:
MAX_LENGTH  = 20
df = labelize_output_according_criterion2(df, map_ohlc, max_trade_length=MAX_LENGTH)

,Time,Open,High,Low,Close,Volume,Open_pct,High_pct,Low_pct,Close_pct,...,VAR_Close_lag_99d,diff_Close_99d,Close_lag_200d,SMA_Close_200d,STD_Close_200d,VAR_Close_lag_200d,diff_Close_200d,TP,SL,signal
0,2021-07-29 00:00:00,39995.453125,40200.531250,39620.410156,39647.917969,0.0,NaN,NaN,NaN,NaN,...,0.000000,0.000000,NaN,39866.078125,0.000000,0.000000,0.000000,40837.355508,39053.199199,0
1,2021-07-29 01:00:00,39604.121094,39728.531250,39352.058594,39685.195312,0.0,-0.978,-1.174,-0.677,0.094,...,37428.907501,-136.800781,NaN,39729.277344,193.465520,37428.907501,-136.800781,40875.751172,39089.917383,0
2,2021-07-29 02:00:00,39684.011719,39942.167969,39655.148438,39764.554688,0.0,0.202,0.538,0.770,0.200,...,19059.924547,21.462240,NaN,39740.008464,138.057686,19059.924547,21.462240,40957.491328,39168.086367,0
3,2021-07-29 03:00:00,39727.890625,39855.910156,39574.535156,39700.894531,0.0,0.111,-0.216,-0.203,-0.160,...,12865.387029,-18.900635,NaN,39733.708252,113.425689,12865.387029,-18.900635,40891.921367,39105.381113,0
4,2021-07-29 04:00:00,39640.609375,40110.203125,39579.765625,39983.476562,4128768.0,-0.220,0.638,0.013,0.712,...,11446.653801,75.844336,NaN,39752.669336,106.989036,11446.653801,75.844336,41182.980859,39383.724414,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17085,2023-07-18 19:00:00,29909.402344,29936.634766,29710.734375,29720.015625,0.0,0.300,0.058,-0.282,-0.608,...,32337.241956,-395.082815,30210.437500,30459.515454,380.401972,144705.660419,-640.318677,30611.616094,29274.215391,0
17086,2023-07-18 20:00:00,29719.406250,29794.064453,29704.521484,29777.044922,0.0,-0.635,-0.476,-0.021,0.192,...,28326.760178,-453.031171,30204.938965,30457.234556,383.292737,146913.321999,-708.475278,30670.356270,29330.389248,0
17087,2023-07-18 21:00:00,29776.722656,29833.054688,29764.291016,29800.257812,0.0,0.193,0.131,0.201,0.078,...,29505.888698,-401.892701,30217.180176,30455.116562,385.788126,148832.477964,-661.535020,30694.265547,29353.253945,0
17088,2023-07-18 22:00:00,29797.849609,29823.927734,29770.392578,29821.093750,0.0,0.071,-0.031,0.020,0.070,...,30998.840751,-389.011403,30268.398926,30452.791147,388.312689,150786.744764,-649.475229,30715.726562,29373.777344,0


In [84]:
#df = my_functions.labelize_output_according_criterion2(df, map_ohlc, max_trade_length=MAX_LENGTH)
#df['signal'] = (df['min_above'].lt(df['min_below'])) & (df['min_above'] <= MAX_LENGTH)
#df['signal']=df['signal'].astype(int)
#df.drop(columns=['CloseAbovethreshold', 'HighAbovethreshold',
#       'LowAbovethreshold', 'OpenAbovethreshold', 'CloseBelowthreshold',
#       'HighBelowthreshold', 'LowBelowthreshold', 'OpenBelowthreshold', 'TP',
#       'SL', 'min_above', 'min_below'], inplace=True, errors='ignore')


features = [col for col in df.columns if col not in ['Time','YM','train_test','signal']]
for feature in features:
     df[feature] = df[feature] - df[feature].shift()
     df[feature] = np.sqrt(np.absolute(df[feature] - df[feature].shift().fillna(0)))*np.sign(df[feature] - df[feature].shift().fillna(0))



df = my_functions.round_float_cols(df)
df.head()

,Time,Open,High,Low,Close,Volume,Open_pct,High_pct,Low_pct,Close_pct,...,VAR_Close_lag_99d,diff_Close_99d,Close_lag_200d,SMA_Close_200d,STD_Close_200d,VAR_Close_lag_200d,diff_Close_200d,TP,SL,signal
0,2021-07-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2021-07-29 01:00:00,-19.78,-21.73,-16.38,6.11,0.00,NaN,NaN,NaN,NaN,...,193.47,-11.70,NaN,-11.70,13.91,193.47,-11.70,6.20,6.06,0
2,2021-07-29 02:00:00,21.71,26.18,23.90,6.49,0.00,1.09,1.31,1.20,0.33,...,-236.22,17.18,NaN,12.15,-15.78,-236.22,17.18,6.58,6.44,0
3,2021-07-29 03:00:00,-6.00,-17.32,-19.59,-11.96,0.00,-1.13,-1.57,-1.56,-0.68,...,110.34,-14.09,NaN,-4.13,5.55,110.34,-14.09,-12.14,-11.87,0
4,2021-07-29 04:00:00,-11.45,18.45,9.27,18.61,2031.94,-0.49,1.27,1.09,1.11,...,69.11,11.62,NaN,5.03,4.27,69.11,11.62,18.88,18.47,0


In [18]:
#from sklearn.preprocessing import MinMaxScaler
#
#scaler = MinMaxScaler(feature_range=(-1,1))
#

In [85]:
df.drop(columns=['TP','SL','Open','Close','High','Low', 'Volume'], inplace=True, errors='ignore')
df = df.dropna()

In [87]:

df_test = df[-cfg['TEST_SIZE']:].reset_index(drop=True)
df_train = df[:-cfg['TEST_SIZE']]
df_val = df_train[-cfg['VALIDATION_SIZE']:].reset_index(drop=True)
df_train = df_train[:-cfg['VALIDATION_SIZE']].reset_index(drop=True)

print(len(df))
print(len(df_train),len(df_val),len(df_test))
print(len(df_train)+len(df_val)+len(df_test))


df_test.to_csv(cfg['DATASET_FOLDER']+'/'+cfg['TEST_FILE'], index=False)
df_train.to_csv(cfg['DATASET_FOLDER']+'/'+cfg['TRAIN_FILE'], index=False)
df_val.to_csv(cfg['DATASET_FOLDER']+'/'+cfg['VALIDATION_FILE'], index=False)


16889
11129 2880 2880
16889
